# Descarga datos esios

## Documentacion:

La nueva web pública de e·sios pone a disposición de todos los usuarios una API para la descarga de información, cuya documentación se encuentra disponible en https://api.esios.ree.es.

Para poder utilizar esta API deberán solicitar un token personal enviando un correo a **consultasios@ree.es**.En mi caso:

token: 9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10

En nuestro caso, la solicitud de datos será para un periodo de tiempo solicitando los datos por código indicador:

https://api.esios.ree.es/indicator/getting_a_specific_indicator_filtering_values_by_a_date_range


## 1. Importamos librerías necesarias

In [96]:
import pandas as pd
import numpy as np
import requests
import json

In [97]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary or bytes to be sent in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      /anaconda3/lib/python3.6/site-packages/requests/api.py
Type:      function


## 2. Definición features

Creamos diccionario

In [98]:
d={'P48_EOLICA':10010,
    'P48_DEMANDA':10027,
    #'POT_DISP_HULLA_ANT':475,  #Los tendremos que sumar para obtener MW disponibles Carbón
    #'POT_DISP_HULLA_SB':476,
    #'PRECIO_MD_ESP':600,
    #'PRECIO_MD_FR':600,
    #'PRECIO_I1':612
  }

In [99]:
d.items()

dict_items([('P48_EOLICA', 10010), ('P48_DEMANDA', 10027)])

En el caso del id_precios habrá que adentrarse un nivel adicional y consultar GEOIDS:

* ESPAÑA = 3
* FRANCIA = 2

## 3. Funciones solicitud datos

Función para obtener los datos del indicador *ind* solicitado entre dos fechas dadas.

En nuestro dataset se nos presentan 2 peculiaridades a introducir en la función:

1) Existe un único indicador para los precios de cada mercado, diferenciándose el pais por el parámetro GEOIDS --> **solicita_datos_precios**

2) Los datos de potencia disponible hay que agregarlos porque viene desglosada en comunidades --> **solicita datos disponibles**

In [104]:
def solicita_datos_general(fecha_ini,fecha_fin,ind):

    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }

    query_parametros_={'start_date': fecha_ini,
                'end_date':fecha_fin}

    url_='https://api.esios.ree.es/indicators/' + str(ind)
    
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
    
    #Comprobamos que trae datos
    assert r.status_code==200
    
    #r.content --> durante el diseño para explorar datos
    
    data=json.loads(r.content)
    
    #type(data)
    #data['indicator']['values'] --> durante el diseño para explorar datos
    
    df_data=pd.DataFrame(data['indicator']['values'])
    
    #df_data.dtypes --> Tenemos que convertir las fechas a formato fecha
    #y crear nuestro df
    
    df=pd.DataFrame()        
    df['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data['datetime']]) 
    df['value']=df_data['value']
    
    #df['period']=pd.Series([int(f[11:13])+1 for f in df_data['datetime']]) La crearemos al final
    
    df=df.set_index('datetime')

    return df

## 4. Agregación dataset

In [105]:
datos_esios=pd.DataFrame()  #será nuestro set de datos

fecha_ini_='2015-10-01 00:00:00'
fecha_fin_='2018-09-30 23:59:59'

Definimos parámetros de entrada:

La idea inicial es que mi dataset empiece en 01/10/2015 hasta 30/09/2018.

He escogido 3 años móviles, teniendo en cuenta que el año hidráulico comienza el octubre (y la hidraulicidad es importante en el Mercado eléctrico) e intentar evitar el efecto de cambios regulatorios en la mayor medida posible

In [107]:
for k,v in d.items():
    historico=solicita_datos(fecha_ini_,fecha_fin_,v)
    historico.columns=[str(k)]
    datos_esios=pd.concat([datos_esios,historico],axis=1)

In [108]:
datos_esios.head()

,P48_EOLICA,P48_DEMANDA
datetime,,
2015-10-01 00:00:00,2196.1,24517.1
2015-10-01 01:00:00,2064.4,23168.6
2015-10-01 02:00:00,1866.5,22437.2
2015-10-01 03:00:00,1660.9,22281.3
2015-10-01 04:00:00,1373.9,21909.7


## 5. Check datos

In [109]:
datos_esios.describe()

,P48_EOLICA,P48_DEMANDA
count,26304.000000,26304.000000
mean,5479.286014,28672.990169
std,3185.927399,4616.496856
min,240.800000,17728.400000
25%,3018.400000,24734.150000
50%,4822.950000,28920.600000
75%,7289.250000,32267.650000
max,16952.600000,41297.900000


In [112]:
datos_esios.shape  #para testear que no tengo NA

(26304, 2)

## 6. Salvar datos